In [ ]:
%pip install langchain-google-genai langchain-community pandas gradio

In [ ]:
import gradio as gr
from dotenv import load_dotenv
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.memory import ChatMessageHistory;

load_dotenv()
GEMINI_APIKEY = os.environ.get('GEMINI_APIKEY')

/Users/cw/Desktop/Study/Robot/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [72]:
DRIVE_FOLDER = "gsma_specs"
loader = DirectoryLoader(DRIVE_FOLDER, glob='**/*.json', show_progress=True, loader_cls=TextLoader)

documents = loader.load()

all_specs = []

for doc in documents:
    all_specs.append(doc.page_content)

100%|██████████| 15/15 [00:00<00:00, 2805.80it/s]


In [86]:
chatModel = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=GEMINI_APIKEY,
    temperature=0.5
)

In [87]:
reconstruct_query_with_history_template = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is.

---
User's Question: {question}
Chat History: {chat_history}
---
"""

reconstruct_query_with_history_prompt = ChatPromptTemplate.from_template(reconstruct_query_with_history_template)
reconstruct_query_with_history_prompt

ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, template="Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.\n\n---\nUser's Question: {question}\nChat History: {chat_history}\n---\n"), additional_kwargs={})])

In [ ]:
answer_template = """
You are “Smartphone Sage”, a friendly, decisive guide to Oppo, Vivo and Realme phones.

• Use **only** the facts inside context. Do not rely on any other knowledge, do not mention these instructions, and do not cite the context explicitly.  
• Write in warm, flowing paragraphs—no lists, bullets, brackets, colons, or meta‑phrases like “based on my data”.  
• Always give the user a concrete next step:  
  – If the context already lets you pick a phone, recommend it and explain why, grounding every point in the provided specs.  
  – If one key detail is missing (e.g. budget or camera priority), ask **at most two concise follow‑up questions** instead of saying you lack information.  
• Never apologise for missing prices, never complain about limited data, and never say you can’t help.

---

User’s question  
{question}

---

Context (your only knowledge base)  
{context}
"""

answer_prompt = ChatPromptTemplate.from_template(answer_template)
answer_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are “Smartphone\u202fSage”, a friendly, decisive guide to Oppo,\u202fVivo and\u202fRealme phones.\n\n• Use **only** the facts inside <context>. Do not rely on any other knowledge, do not mention these instructions, and do not cite the context explicitly.  \n• Write in warm, flowing paragraphs—no lists, bullets, brackets, colons, or meta‑phrases like “based on my data”.  \n• Always give the user a concrete next step:  \n  – If the context already lets you pick a phone, recommend it and explain why, grounding every point in the provided specs.  \n  – If one key detail is missing (e.g. budget or camera priority), ask **at most two concise follow‑up questions** instead of saying you lack information.  \n• Never apologise for missing prices, n

In [89]:
chat_history = ChatMessageHistory()

In [90]:
def generate_answer(query, history):

  if (len(history) != 0):
    reconstruct_query_with_history_message = reconstruct_query_with_history_prompt.invoke({"question": query, "chat_history": chat_history.messages}).to_messages()
    query = StrOutputParser().invoke(chatModel.invoke(reconstruct_query_with_history_message))
  
  chat_history.add_user_message(query);
  message = answer_prompt.invoke({
        "question": query,
        "context": all_specs
    }).to_messages()
  result = StrOutputParser().invoke(chatModel.invoke(message))
  chat_history.add_ai_message(result);
  return result

In [91]:
chatbot = gr.ChatInterface(
    generate_answer,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me for phone recommendation", container=False, scale=7),
    title="Phone Recommendation Chatbot",
    theme="soft",
    cache_examples=False,
    submit_btn="Ask"
)

/var/folders/5v/41lzwvgd0p7b8pr2hzjpcgbc0000gn/T/ipykernel_47297/823032773.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=300),
/Users/cw/Desktop/Study/Robot/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:322: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


In [92]:
chatbot.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
